In [ ]:
pip install requests beautifulsoup4 tqdm pdfminer.six pymupdf

### 라이브러리

In [177]:
import os
import re
import time
from urllib.parse import urljoin
import requests
import pandas as pd
import fitz
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

### 고정 변수 할당

In [161]:
BASE = "https://finance.naver.com"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.7,en;q=0.6",
    "Connection": "keep-alive",
    "Referer": BASE,
})


In [162]:
BASE_DIR = r"C:\Users\User\Desktop\kdt\pjt_bok_1"
PDF_DIR = os.path.join(BASE_DIR, "pdf")
TXT_DIR = os.path.join(BASE_DIR, "text")
DEBUG_DIR = os.path.join(BASE_DIR, "debug_html")
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(TXT_DIR, exist_ok=True)
os.makedirs(DEBUG_DIR, exist_ok=True)


In [175]:
BROKER_CODE = "18"
DATE_FROM = "2015-01-01"
DATE_TO = "2025-12-30"
SEARCH_TYPE = "writeDate"
SLEEP_SEC = 0.3
MAX_PAGE = 260


### 세션 열기 함수

In [179]:
#새로운

DEFAULT_HEADERS = {
    "User-Agent": "Mozilla/5.0",
}

def fetch_html(url: str, referer: str | None = None, max_retries: int = 6) -> str:
    headers = DEFAULT_HEADERS.copy()
    if referer:
        headers["Referer"] = referer

    last_err = None

    for attempt in range(1, max_retries + 1):
        try:
            r = session.get(
                url,
                headers=headers,
                timeout=(10, 25),   # (connect, read)
                allow_redirects=True,
            )
            r.raise_for_status()

            # 인코딩은 강제 euc-kr 대신 자동 추정이 더 안전함
            r.encoding = r.apparent_encoding
            return r.text

        except (requests.exceptions.ConnectionError,
                requests.exceptions.Timeout,
                requests.exceptions.ChunkedEncodingError,
                requests.exceptions.HTTPError) as e:
            last_err = e

            # 지수 백오프 + 랜덤 지터(과부하/차단 회피에 효과적)
            sleep_sec = min(60, (2 ** attempt)) + random.uniform(0.2, 1.2)
            time.sleep(sleep_sec)

    # 여기까지 오면 완전 실패
    raise last_err


In [164]:
def fetch_html(url: str, referer: str | None = None) -> str:
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    if referer:
        headers["Referer"] = referer

    r = session.get(
        url,
        headers=headers,
        timeout=25,
        allow_redirects=True,
    )
    r.raise_for_status()

    r.encoding = r.apparent_encoding
    return r.text


### url 리스트 받는 함수

In [165]:
def get_list_url(page: int) -> str:
    return (
        f"{BASE}/research/debenture_list.naver?"
        f"keyword=&brokerCode={BROKER_CODE}"
        f"&searchType={SEARCH_TYPE}"
        f"&writeFromDate={DATE_FROM}&writeToDate={DATE_TO}"
        f"&x=0&y=0&page={page}"
    )


### nid(고유키) 받는 함수

In [166]:
def get_list_nids(page: int) -> list[int]:
    url = get_list_url(page)
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")

    nids = []
    for a in soup.select('a[href*="debenture_read.naver?nid="]'):
        href = a.get("href", "")
        m = re.search(r"nid=(\d+)", href)
        if m:
            nids.append(int(m.group(1)))

    uniq, seen = [], set()
    for x in nids:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq

### 상세페이지 parse 하기 함수

In [167]:
def parse_detail_page_raw(nid: int, debug: bool = False) -> dict | None:
    list_url = get_list_url(1)
    detail_url = f"{BASE}/research/debenture_read.naver?nid={nid}"

    html = fetch_html(detail_url, referer=list_url)
    soup = BeautifulSoup(html, "lxml")
    text_all = soup.get_text(" ", strip=True)

    og = soup.select_one('meta[property="og:title"]')
    title = og.get("content").strip() if og and og.get("content") else None

    source_p = soup.select_one("p.source")
    source_raw = source_p.get_text(" ", strip=True) if source_p else None

    date = None
    org = None
    if source_raw:
        m_date = re.search(r"(20\d{2}\.\d{2}\.\d{2})", source_raw)
        if m_date:
            date = m_date.group(1)
            org = source_raw.split(date)[0].replace("|", " ").strip()
            org = re.sub(r"\s+", " ", org)

    page_text = None
    td_node = soup.select_one("#contentarea_left td.view_cnt")
    if td_node:
        ps = td_node.select("p")
        if ps:
            page_text = "\n".join(
                p.get_text(" ", strip=True) for p in ps if p.get_text(strip=True)
            )
        else:
            page_text = td_node.get_text("\n", strip=True)

    if not page_text:
        selector = "#contentarea_left > div.box_type_m.box_type_m3 > table > tbody > tr:nth-child(3) > td > div:nth-child(1) > p"
        p_node = soup.select_one(selector)
        if p_node:
            page_text = p_node.get_text("\n", strip=True)

    if page_text:
        page_text = re.sub(r"\n{3,}", "\n\n", page_text).strip()

    if title == "네이버페이 증권" and not date and not page_text:
        if debug:
            with open(os.path.join(DEBUG_DIR, f"landing_{nid}.html"), "w", encoding="utf-8") as f:
                f.write(html)
        return None

    if not (title or source_raw or page_text):
        if debug:
            with open(os.path.join(DEBUG_DIR, f"parse_fail_{nid}.html"), "w", encoding="utf-8") as f:
                f.write(html)
        return None

    pdf_url = None
    for a in soup.select("a[href]"):
        if "btn_report.gif" in str(a):
            pdf_url = urljoin(BASE, a.get("href"))
            break

    return {
        "nid": nid,
        "title": title,
        "org": org,
        "date": date,
        "source_raw": source_raw,
        "page_text": page_text,
        "pdf_url": pdf_url,
    }


### 파일 이름 안정화 함수

In [168]:
def safe_filename(s: str, maxlen: int = 180) -> str:
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:maxlen]


### pdf 다운로드 함수

In [169]:
def download_pdf(meta: dict) -> str:
    filename = f"{meta['date']}_{meta.get('org') or 'UNKNOWN'}_{meta['nid']}.pdf"
    filename = safe_filename(filename)
    path = os.path.join(PDF_DIR, filename)

    if os.path.exists(path) and os.path.getsize(path) > 0:
        return path

    r = session.get(meta["pdf_url"], timeout=60)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)
    return path


### pdf에서 text 추출 함수

In [170]:
def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text() for page in doc)
    doc.close()
    return text


### text를 csv 파일에 저장

In [171]:
def save_pdf_text(meta: dict, pdf_path: str, pdf_text: str) -> str:
    txt_name = os.path.basename(pdf_path).replace(".pdf", ".txt")
    txt_path = os.path.join(TXT_DIR, txt_name)
    if not os.path.exists(txt_path):
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(pdf_text)
    return txt_path


### 네이버 증권 페이지를 df로 변환하는 함수 

In [172]:
def crawl_all_page_only_to_df() -> pd.DataFrame:
    rows = []
    page = 1
    empty_pages = 0
    done = set()
    
    while True:
        if page > MAX_PAGE:
            break

        nids = get_list_nids(page)

        if not nids:
            empty_pages += 1
            if empty_pages >= 2:
                break
            page += 1
            continue
        
        empty_pages = 0
        for nid in tqdm(nids, desc=f"page {page}"):
            if nid in done:
                continue

            meta = parse_detail_page_raw(nid, debug=True)
            done.add(nid)
            if meta is None:
                continue

            rows.append({
                "nid": meta["nid"],
                "date": meta["date"],
                "org": meta["org"],
                "title": meta["title"],
                "source_raw": meta["source_raw"],
                "page_text": meta["page_text"],
            })

            time.sleep(SLEEP_SEC)

        page += 1

    return pd.DataFrame(rows)


### pdf text를 df 로 변환하는 함수

In [173]:
def crawl_all_pdf_to_df(include_pdf_text: bool = True) -> pd.DataFrame:
    rows = []
    page = 1
    empty_pages = 0
    done = set()

    while True:
        nids = get_list_nids(page)
        if not nids:
            empty_pages += 1
            if empty_pages >= 2:
                break
            page += 1
            continue

        empty_pages = 0
        for nid in tqdm(nids, desc=f"page {page}"):
            if nid in done:
                continue

            meta = parse_detail_page_raw(nid, debug=True)
            done.add(nid)
            if meta is None or not meta.get("pdf_url"):
                continue

            pdf_path = download_pdf(meta)
            pdf_text = extract_text_from_pdf(pdf_path)

            txt_path = save_pdf_text(meta, pdf_path, pdf_text)

            row = {
                "nid": meta["nid"],
                "date": meta["date"],
                "org": meta["org"],
                "title": meta["title"],
                "pdf_url": meta["pdf_url"],
                "pdf_path": pdf_path,
                "txt_path": txt_path,
            }
            if include_pdf_text:
                row["pdf_text"] = pdf_text

            rows.append(row)
            time.sleep(SLEEP_SEC)

        page += 1

    return pd.DataFrame(rows)


### df를 csv로 저장하는 함수

In [174]:
import csv

def save_df_to_csv(df, path):
    df.to_csv(
        path,
        index=False,
        encoding="utf-8-sig",
        quoting=csv.QUOTE_ALL,
        lineterminator="\n"
    )


### 크롤링한 페이지 정보를 csv로 저장

In [180]:
df_page = crawl_all_page_only_to_df()

df_page["page_text"] = (
    df_page["page_text"]
    .fillna("")
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

save_df_to_csv(df_page, os.path.join(BASE_DIR, "naver_debenture_page_text.csv"))


page 260: 100%|██████████| 8/8 [00:03<00:00,  2.48it/s]


### 크롤링한 pdf 정보를 csv로 저장

In [ ]:
df_pdf = crawl_all_pdf_to_df(include_pdf_text=True)
save_df_to_csv(df_pdf, os.path.join(BASE_DIR, "naver_debenture_pdf_text.csv"))


: 